In [1]:
import numpy as np
from dipy.viz import actor, window, ui

In [2]:
from dipy.data.fetcher import fetch_bundles_2_subjects, read_bundles_2_subjects
fetch_bundles_2_subjects()

Data size is approximately 234MB
Dataset is already in place. If you want to fetch it again please first remove the folder /home/romain/.dipy/exp_bundles_and_maps 


({'bundles_2_subjects.tar.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/38477/bundles_2_subjects.tar.gz',
   '97756fbef11ce2df31f1bedf1fc7aac7')},
 '/home/romain/.dipy/exp_bundles_and_maps')

In [3]:
res = read_bundles_2_subjects('subj_1', ['t1', 'fa'],
                              ['af.left', 'cst.right', 'cc_1'])

In [4]:
streamlines = res['af.left'] + res['cst.right'] + res['cc_1']
data = res['fa']
shape = data.shape
affine = res['affine']

In [5]:
world_coords = True

In [6]:
if not world_coords:
    from dipy.tracking.streamline import transform_streamlines
    streamlines = transform_streamlines(streamlines, np.linalg.inv(affine))

In [7]:
ren = window.Renderer()
stream_actor = actor.line(streamlines)

if not world_coords:
    image_actor_z = actor.slicer(data, affine=np.eye(4))
else:
    image_actor_z = actor.slicer(data, affine)

In [8]:
slicer_opacity = 0.6
image_actor_z.opacity(slicer_opacity)

In [9]:
image_actor_x = image_actor_z.copy()
image_actor_x.opacity(slicer_opacity)
x_midpoint = int(np.round(shape[0] / 2))
image_actor_x.display_extent(x_midpoint,
                             x_midpoint, 0,
                             shape[1] - 1,
                             0,
                             shape[2] - 1)

image_actor_y = image_actor_z.copy()
image_actor_y.opacity(slicer_opacity)
y_midpoint = int(np.round(shape[1] / 2))
image_actor_y.display_extent(0,
                             shape[0] - 1,
                             y_midpoint,
                             y_midpoint,
                             0,
                             shape[2] - 1)

In [10]:
ren.add(stream_actor)
ren.add(image_actor_z)
ren.add(image_actor_x)
ren.add(image_actor_y)

In [11]:
show_m = window.ShowManager(ren, size=(1200, 900))
show_m.initialize()

In [12]:
line_slider_z = ui.LineSlider2D(min_value=0,
                                max_value=shape[2] - 1,
                                initial_value=shape[2] / 2,
                                text_template="{value:.0f}")

line_slider_x = ui.LineSlider2D(min_value=0,
                                max_value=shape[0] - 1,
                                initial_value=shape[0] / 2,
                                text_template="{value:.0f}")

line_slider_y = ui.LineSlider2D(min_value=0,
                                max_value=shape[1] - 1,
                                initial_value=shape[1] / 2,
                                text_template="{value:.0f}")

opacity_slider = ui.LineSlider2D(min_value=0.0,
                                 max_value=1.0,
                                 initial_value=slicer_opacity)

In [13]:
def change_slice_z(i_ren, obj, slider):
    z = int(np.round(slider.value))
    image_actor_z.display_extent(0, shape[0] - 1, 0, shape[1] - 1, z, z)


def change_slice_x(i_ren, obj, slider):
    x = int(np.round(slider.value))
    image_actor_x.display_extent(x, x, 0, shape[1] - 1, 0, shape[2] - 1)


def change_slice_y(i_ren, obj, slider):
    y = int(np.round(slider.value))
    image_actor_y.display_extent(0, shape[0] - 1, y, y, 0, shape[2] - 1)


def change_opacity(i_ren, obj, slider):
    slicer_opacity = slider.value
    image_actor_z.opacity(slicer_opacity)
    image_actor_x.opacity(slicer_opacity)
    image_actor_y.opacity(slicer_opacity)

line_slider_z.add_callback(line_slider_z.slider_disk,
                           "MouseMoveEvent",
                           change_slice_z)
line_slider_x.add_callback(line_slider_x.slider_disk,
                           "MouseMoveEvent",
                           change_slice_x)
line_slider_y.add_callback(line_slider_y.slider_disk,
                           "MouseMoveEvent",
                           change_slice_y)
opacity_slider.add_callback(opacity_slider.slider_disk,
                            "MouseMoveEvent",
                            change_opacity)

In [14]:
line_slider_label_z = ui.TextBox2D(text="Z Slice", width=50, height=20)
line_slider_label_x = ui.TextBox2D(text="X Slice", width=50, height=20)
line_slider_label_y = ui.TextBox2D(text="Y Slicer", width=50, height=20)
opacity_slider_label = ui.TextBox2D(text="Opacity", width=50, height=20)

In [15]:
panel = ui.Panel2D(center=(1030, 120),
                   size=(300, 200),
                   color=(1, 1, 1),
                   opacity=0.1,
                   align="right")

panel.add_element(line_slider_label_x, 'relative', (0.1, 0.8))
panel.add_element(line_slider_x, 'relative', (0.5, 0.8))
panel.add_element(line_slider_label_y, 'relative', (0.1, 0.6))
panel.add_element(line_slider_y, 'relative', (0.5, 0.6))
panel.add_element(line_slider_label_z, 'relative', (0.1, 0.4))
panel.add_element(line_slider_z, 'relative', (0.5, 0.4))
panel.add_element(opacity_slider_label, 'relative', (0.1, 0.2))
panel.add_element(opacity_slider, 'relative', (0.5, 0.2))

show_m.ren.add(panel)

In [ ]:
global size
size = ren.GetSize()


def win_callback(obj, event):
    global size
    if size != obj.GetSize():
        size_old = size
        size = obj.GetSize()
        size_change = [size[0] - size_old[0], 0]
        panel.re_align(size_change)

show_m.initialize()

In [ ]:
show_m.add_window_callback(win_callback)
show_m.render()
show_m.start()

#ren.zoom(1.5)
#ren.reset_clipping_range()

#window.record(ren, out_path='bundles_and_a_slice.png', size=(1200, 900),reset_camera=False)

In [ ]:
from dipy.viz import fvtk
r=fvtk.ren()
a=fvtk.axes()
fvtk.add(r,a)
fvtk.show(r)